# GA & GP optimization of NN

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

import genetic_algorithm as ga
from nn_ga import GenAlgNN
from nn_gp import GenProgNN

## Generate and load data

In [ ]:
from math import pi, sin, cos
theta = 0.5 * pi
while theta < 3.5 * pi:
    r = theta * 1.5 / pi
    x = r * sin(theta)
    y = r * cos(theta)
    print(f'{x:.6f},{y:.6f},1')
    print(f'{-x:.6f},{-y:.6f},-1')
    theta += 0.125 / r

In [ ]:
spiral = pd.read_csv('../two_spirals.csv', names=['x', 'y', 'class'])

In [ ]:
X_valid = spiral.copy()
y_valid = X_valid.pop('class')

X_train = spiral.sample(frac=0.5)
y_train = X_train.pop('class')

In [ ]:
X_valid

In [ ]:
y_valid

In [ ]:
X_train

In [ ]:
y_train

## Visualisation utilities

In [ ]:
def visualize_model(model):
    global X_train, y_train
    points = np.zeros([120, 120, 2])
    for y in range(120):
        for x in range(120):
            points[y, x, 0] = (x - 60) / 10.0
            points[y, x, 1] = (y - 60) / 10.0
    plt.imshow(model.predict(points.reshape([-1, 2])).reshape([120, 120]), cmap='cividis', vmin=-1, vmax=1)
    plt.colorbar()
    plt.axis('off')
    points = X_train.where(y_train > 0) * 10 + 60
    plt.scatter(points['x'], points['y'], c='red')
    points = X_train.where(y_train < 0) * 10 + 60
    plt.scatter(points['x'], points['y'], c='limegreen')
    plt.show()

In [ ]:
def print_population(fitnesses, population):
    arr = [(1.0 / f - 1.0, i) for f, i in zip(fitnesses, population)]
    arr.sort(key=lambda t: t[0])
    for m, i in arr:
        print(f'mse={m:.4f} {i}')
    return arr

## Test NN

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(6, activation='tanh'),
    keras.layers.Dense(4, activation='tanh'),
    keras.layers.Dense(1, activation='tanh')])
model.compile(loss='mse', optimizer=keras.optimizers.SGD(lr=0.03))
model.fit(np.array(X_train), np.array(y_train), epochs=2000, batch_size=8, verbose=0)
keras.losses.MeanSquaredError()(y_valid, model.predict(X_valid)).numpy()

In [ ]:
visualize_model(model)

## Genetic Algorithm

In [ ]:
problem_ga = GenAlgNN((np.array(X_train), np.array(y_train)), (np.array(X_valid), np.array(y_valid)))
ga_optimizer = ga.GAOptimizer(problem_ga)

In [ ]:
ga_optimizer.start(8)
for i in range(6):
    old_population = ga_optimizer.population
    gen, old_fitnesses = ga_optimizer.next_generation()
    print_population(old_fitnesses, old_population)
    print('=== New generation ===', i)

In [ ]:
fitnesses = [problem_ga.fitness(i) for i in gen]
mse = print_population(fitnesses, gen)

In [ ]:
problem_ga.epochs = 4000
model = problem_ga.train_nn(mse[0][1])
keras.losses.MeanSquaredError()(y_valid, model.predict(X_valid)).numpy()

In [ ]:
visualize_model(model)

## Genetic Programming

In [ ]:
problem_gp = GenProgNN((np.array(X_train), np.array(y_train)), (np.array(X_valid), np.array(y_valid)))
gp_optimizer = ga.GAOptimizer(problem_gp)

In [ ]:
gp_optimizer.start(10)
for i in range(8):
    old_population = gp_optimizer.population
    gen, old_fitnesses = gp_optimizer.next_generation()
    print_population(old_fitnesses, old_population)
    print('=== New generation ===', i)

In [ ]:
fitnesses = [problem_gp.fitness(i) for i in gen]
mse = print_population(fitnesses, gen)

In [ ]:
problem_gp.epochs = 4000
model = problem_gp.train_nn(mse[0][1])
keras.losses.MeanSquaredError()(y_valid, model.predict(X_valid)).numpy()

In [ ]:
visualize_model(model)

In [ ]:
for item in mse:
    print('digraph {')
    print(item[1].to_dot())
    print('}')
    print()